In [ ]:
import pandas as pd
train=pd.read_csv('../input/loan-default-prediction/train_LZV4RXX.csv')
train.head()

In [ ]:
test=pd.read_csv('../input/loan-default-prediction/test_4zJg83n.csv')
test.head()

In [ ]:
train.nunique()

In [ ]:
train.info()

In [ ]:
train.proof_submitted.value_counts()

In [ ]:
round(train.isnull().sum()*100/len(train),2)

In [ ]:
round(test.isnull().sum()*100/len(test),2)

In [ ]:
train=train.drop(['loan_id'],axis=1)
train.head()

In [ ]:
test=test.drop(['loan_id'],axis=1)
test.head()

In [ ]:
train[['education','last_delinq_none']]=train[['education','last_delinq_none']].astype('object')
train.info()

In [ ]:
test[['education','last_delinq_none']]=test[['education','last_delinq_none']].astype('object')
test.info()

In [ ]:
!pip install flaml
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
train=pd.get_dummies(train,prefix_sep='__')
train.head()

In [ ]:
test=pd.get_dummies(test,prefix_sep='__')
test.head()

In [ ]:
# !rm -r kuma_utils
!git clone https://github.com/analokmaus/kuma_utils.git

In [ ]:
import sys
sys.path.append("kuma_utils/")
from kuma_utils.preprocessing.imputer import LGBMImputer

In [ ]:
col=train.columns.tolist()
col.remove('loan_default')
col[:5]

In [ ]:
%%time
lgbm_imtr = LGBMImputer(n_iter=500)

train_iterimp = lgbm_imtr.fit_transform(train[col])
test_iterimp = lgbm_imtr.transform(test[col])

# Create train test imputed dataframe
train_ = pd.DataFrame(train_iterimp, columns=col)
test = pd.DataFrame(test_iterimp, columns=col)

In [ ]:
train_['loan_default'] = train['loan_default']
train_.head()

In [ ]:
def undummify(df, prefix_sep="__"):
    cols2collapse = {
        item.split(prefix_sep)[0]: (prefix_sep in item) for item in df.columns
    }
    series_list = []
    for col, needs_to_collapse in cols2collapse.items():
        if needs_to_collapse:
            undummified = (
                df.filter(like=col)
                .idxmax(axis=1)
                .apply(lambda x: x.split(prefix_sep, maxsplit=1)[1])
                .rename(col)
            )
            series_list.append(undummified)
        else:
            series_list.append(df[col])
    undummified_df = pd.concat(series_list, axis=1)
    return undummified_df


In [ ]:
train=undummify(train_)
train.head()

In [ ]:
test=undummify(test)
test.head()

In [ ]:
from flaml import AutoML
automl = AutoML()

In [ ]:
y = train.pop('loan_default')
X = train

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42,shuffle=True, stratify=y)

In [ ]:
automl.fit(X_train, y_train, task="classification",metric='micro_f1',time_budget=900)

In [ ]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best micro_f1 on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_train, automl.predict(X_train)))

In [ ]:
print(classification_report(y_test, automl.predict(X_test)))

In [ ]:
y_pred = automl.predict(test)
y_pred[:5]

In [ ]:
df = pd.DataFrame(y_pred,columns=['loan_default'])
df.head()

In [ ]:
sol=pd.read_csv('../input/loan-default-prediction/sample_submission_tbPU9qQ.csv')
sol.head()

In [ ]:
sol['loan_default']=df['loan_default']
sol.head()

In [ ]:
sol.to_csv('./micro_f1.csv',index=False)